In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder

# ==========================================
# 1. LOAD DATA & PREPROCESSING
# ==========================================
print("Sedang memproses data...")

df = pd.read_csv('Latest 2025 movies Datasets.csv')

# Membersihkan data: pastikan kolom penting tidak kosong
required_cols = ['release_date', 'vote_average', 'popularity', 'vote_count', 'original_language']
df = df.dropna(subset=required_cols)

# Konversi release_date ke datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df = df.dropna(subset=['release_date'])  # hapus yang gagal konversi

# Feature Engineering
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

# Encoding original_language
le = LabelEncoder()
df['original_language_encoded'] = le.fit_transform(df['original_language'])

# Menentukan Fitur & Target
features = ['popularity', 'vote_count', 'release_year', 'release_month', 'original_language_encoded']
X = df[features]
y = df['vote_average']

# Split Data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ==========================================
# 2. TRAINING MODEL (DECISION TREE)
# ==========================================
print("Sedang melatih model Decision Tree...")

model = DecisionTreeRegressor(max_depth=5, random_state=42)
model.fit(X_train, y_train)

# ==========================================
# 3. EVALUASI LENGKAP & CROSS VALIDATION
# ==========================================
print("\n" + "="*40)
print("HASIL EVALUASI (DECISION TREE)")
print("="*40)

# A. Evaluasi Single Split (Test Set)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"1. Single Split Test:")
print(f"   - R2 Score (Akurasi) : {r2:.4f} ({r2*100:.2f}%)")
print(f"   - RMSE (Error Kuadrat): {rmse:.4f}")
print(f"   - MAE (Rata-rata Error): {mae:.4f} poin")

# B. Cross Validation (5-Fold)
print(f"\n2. Cross Validation (5-Fold):")
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print(f"   - Skor per fold  : {cv_scores}")
print(f"   - Rata-rata R2   : {cv_scores.mean():.4f}")
print(f"   - Kestabilan     : +/- {cv_scores.std():.4f}")

# C. Contoh Prediksi
print("\n" + "="*40)
sample_index = y_test.index[0]  # pastikan index asli
print(f"Contoh Prediksi:")
print(f" - Rating Asli     : {y_test.loc[sample_index]}")
print(f" - Prediksi Model  : {y_pred[0]:.2f}")


Sedang memproses data...
Sedang melatih model Decision Tree...

HASIL EVALUASI (DECISION TREE)
1. Single Split Test:
   - R2 Score (Akurasi) : 0.8059 (80.59%)
   - RMSE (Error Kuadrat): 1.0580
   - MAE (Rata-rata Error): 0.7046 poin

2. Cross Validation (5-Fold):
   - Skor per fold  : [0.5092657  0.74560943 0.78916584 0.80808243 0.81677625]
   - Rata-rata R2   : 0.7338
   - Kestabilan     : +/- 0.1149

Contoh Prediksi:
 - Rating Asli     : 7.0
 - Prediksi Model  : 6.19
